# Create simple neural sequence and save timestamps and activity pattern to file

In [1]:
%reset -f
from anim import build_frames, merge_frames, create_mp4
from copy import copy
import numpy as np
import os

from ntwk import NtwkResponse
from aux import save
from aux import save_time_file

DUR = 2
FS = 1000
V_REST = -0.07
V_TH = -0.04

STARTS = [100, 300, 500, 700]
ITVL = 200

ts = np.linspace(0, DUR, FS * DUR, endpoint=False)
vs = V_REST * np.ones((len(ts), len(STARTS)))
spks = np.zeros(vs.shape, dtype=bool)

for neur, start in enumerate(STARTS):
    for offset in [0, 1000]:
        vs[offset+start:offset+start+ITVL, neur] = np.linspace(V_REST, V_TH, ITVL)
        spks[offset+start+ITVL, neur] = True
        
xs = np.arange(len(STARTS))
ys = np.arange(len(STARTS))

place_field_centers = np.array([xs, ys])

# save everything
save_time_file('test_anim/time.npy', ts)
ntwk_response = NtwkResponse(
    vs, spks, V_REST, V_TH,
    gs=None, ws_rcr=None, ws_up=None, place_field_centers=place_field_centers)
ntwk_response.save('test_anim/ntwk.npy')

'test_anim/ntwk.npy'

### Build frames and gif

In [2]:
FPS = [30, 120, 60, 30]
PLAYBACK_FPS = [15, 30, 30, 30]

In [3]:
for fps, playback_fps in zip(FPS, PLAYBACK_FPS):
    save_file_movie = 'test_anim/ntwk_{}_fps_{}Hz_playback'.format(
        fps, playback_fps)

    save_files = build_frames.ntwk_activity(
        save_prefix='test_anim/ntwk_frames/frame',
        time_file='test_anim/time.npy',
        ntwk_file='test_anim/ntwk.npy',
        fps=fps,
        frames_per_spk=5,
        title='Network activity',
        x_label='x (m)',
        y_label='y (m)',
        fig_size=(7, 7),
        verbose=True)
    
    movie = create_mp4(
        save_files, save_file=save_file_movie,
        playback_fps=playback_fps, verbose=True)

save_files_ntwk = copy(save_files)
movie_ntwk = movie

Using timestamp file "test_anim/time.npy" and activity file "test_anim/ntwk.npy".
Frames will be saved with prefix "test_anim/ntwk_frames/frame".
Loading timestamps and network activity data...
Data loaded.
Downsampling data from 1000.0 Hz to 30 fps...
Data downsampled.
Generating and saving 60 frames...
Frames saved.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/ntwk_30_fps_15Hz_playback.mp4"...
Movie generated.
Using timestamp file "test_anim/time.npy" and activity file "test_anim/ntwk.npy".
Frames will be saved with prefix "test_anim/ntwk_frames/frame".
Loading timestamps and network activity data...
Data loaded.
Downsampling data from 1000.0 Hz to 120 fps...
Data downsampled.
Generating and saving 240 frames...
Frames saved.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/ntwk_120_fps_30Hz_playback.mp4"...
Movie generated.
Using timestamp file "test_anim/time.npy" and activity file "test_anim/ntwk.npy".
Frames will be saved with prefix "test_anim/ntwk_frames/frame".
Loading timestamps and network activity data...
Data loaded.
Downsampling data from 1000.0 Hz to 60 fps...
Data downsampled.
Generating and saving 120 frames...
Frames saved.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/ntwk_60_fps_30Hz_playback.mp4"...
Movie generated.
Using timestamp file "test_anim/time.npy" and activity file "test_anim/ntwk.npy".
Frames will be saved with prefix "test_anim/ntwk_frames/frame".
Loading timestamps and network activity data...
Data loaded.
Downsampling data from 1000.0 Hz to 30 fps...
Data downsampled.
Generating and saving 60 frames...
Frames saved.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/ntwk_30_fps_30Hz_playback.mp4"...
Movie generated.


# Create position sequence for trajectory animation

In [4]:
from scipy.ndimage import gaussian_filter1d

STD = 1
SMOOTHING = 200
SEED = 0

np.random.seed(SEED)

# make random x and y position sequences
xys = STD * np.random.randn(len(ts), 2)
# smooth them
xys = gaussian_filter1d(xys, SMOOTHING, axis=0)

# make sure save directory exists
if not os.path.exists('test_anim'): os.makedirs('test_anim')

# save traj file
data_tr = {'xys': xys}
save('test_anim/traj.npy', data_tr)

'test_anim/traj.npy'

### Build frames and MP4

In [5]:
for fps, playback_fps in zip(FPS, PLAYBACK_FPS):
    save_file_movie = 'test_anim/traj_{}_fps_{}Hz_playback'.format(
        fps, playback_fps)

    save_files = build_frames.traj(
        save_prefix='test_anim/traj_frames/frame',
        time_file='test_anim/time.npy',
        traj_file='test_anim/traj.npy',
        fps=fps,
        title='Trajectory through space',
        x_label='x (m)',
        y_label='y (m)',
        fig_size=(7, 7),
        verbose=True)

    movie = create_mp4(
        save_files, save_file=save_file_movie,
        playback_fps=playback_fps, verbose=True)

save_files_traj = copy(save_files)
movie_traj = movie

Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/traj_30_fps_15Hz_playback.mp4"...
Movie generated.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/traj_120_fps_30Hz_playback.mp4"...
Movie generated.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/traj_60_fps_30Hz_playback.mp4"...
Movie generated.
Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/traj_30_fps_30Hz_playback.mp4"...
Movie generated.


# Create merged frames and MP4

In [6]:
save_prefix = 'test_anim/merged_frames/frame'
rects = [(0, 0, 1, 1), (1, 0, 2, 1)]
size = (2, 1)

merged_frames = merge_frames([save_files_ntwk, save_files_traj], save_prefix, rects, size)

movie_merged = create_mp4(merged_frames, 'test_anim/merged', playback_fps=30, verbose=True)

Loading source images...


Source images loaded.
Generating movie and saving at "test_anim/merged.mp4"...
Movie generated.
